# Модуль 12. **Основи NLP** 

### Завдання

Зробіть summary нижчевказаного тексту використовуючи бібліотеки для NLP: nltk та SpaCy

In [26]:
# ! python.exe -m pip install --upgrade pip
! pip install -U pip setuptools wheel
! pip install -U spacy

In [23]:
import spacy

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

import string
from heapq import nlargest

In [24]:
nlp = spacy.load("en_core_web_sm")

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
# Текст для обробки
text = "This is an example sentence for tokenization and lemmatization."

# Токенізація
doc = nlp(text)
tokens = [token.text for token in doc]
print(tokens)

In [ ]:
tokens = word_tokenize(text)
sentences = sent_tokenize(text)
stop_words = set(stopwords.words("english"))

In [ ]:
punctuation = punctuation + "\n"

In [ ]:
word_frequencies = {}
for word in doc:
    if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1

In [ ]:
select_length = int(len(sentence_tokens))
summary = nlargest(select_length, sentence_scores, key=sentence_scores.get)
summary

<div class="alert alert-block alert-info">

#### **Висновки:**

#### Найкращу точність на тестових даних показала модель *Gated Recurrent Unit (GRU)* - майже **90%**
#### На другому місці, з результатом **89.16%** - модель *Long Short-Term Memory (LSTM)*
#### Найгірша точність у моделі *Simple Recurrent Neural Network (RNN)* - **81,14%**
#### Всі моделі демонструюсь високу точність (>92%) на тренувальних даних, при цьому дуже швидко настає перенавчання, яке вдалося трохи зменшити за рахунок _dropout'у_ та _skip_top_
#### Модель *Deep Recurrent Neural Network (DRNN)* навчається в 2+ рази довше за решту, при цьому результат видає середній 

</div>